# Task 4: Demonstrating Quantum Advantage

In [ ]:
"""
This task is intentionally open-ended, so no boilerplate code is provided.

You may use this notebook to develop your solution, or create a separate file if you prefer.
We recommend starting by copying over your previous implementations of the QRNG, TRNG, and PRNG.
Then, explore ways to modularize and combine these components to design a use case that
demonstrates the unique advantages of quantum randomness.

Your write-up can be included directly in this notebook or submitted separately.
You're welcome to prepare it as a Google Doc or LaTeX document and upload a PDF to the GitHub repository—
just be sure to clearly indicate where it can be found if it's not included here.
"""

### Necessary Imports

In [ ]:
import requests
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
from collections import Counter

### Template for using api

In [4]:
import requests
from typing import Union, List, Optional

QRNG_API_URL = "https://qrng.idqloud.com/api/1.0"
class QRNGaaS:
    def __init__(self, api_key: str):
        self.base_url = QRNG_API_URL
        self.headers = {
            "X-API-KEY": api_key,
            "Content-Type": "application/json"
        }
        self.max_batch_size = 32  # Actual API limit (32 for int, 64 for short)
        self.max_double_batch = 16
    
    def get_random_ints_batched(self, quantity: int) -> List[int]:
        """Get random 32-bit integers with batching to handle API limits"""
        batches = quantity // self.max_batch_size
        remainder = quantity % self.max_batch_size
        
        results = []
        for _ in range(batches):
            results.extend(self._get_batch(self.max_batch_size))
        
        if remainder > 0:
            results.extend(self._get_batch(remainder))
            
        return results
    
    def _get_batch(self, batch_size: int) -> List[int]:
        """Get a single batch of random integers"""
        params = {"quantity": batch_size}
        try:
            response = requests.get(
                f"{self.base_url}/int",
                headers=self.headers,
                params=params,
                timeout=10  # Add timeout to prevent hanging
            )
            response.raise_for_status()
            return response.json()["data"]
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return []  # Return empty list on failure
        
    def _get_double_batch(self, batch_size: int, min_val: float, max_val: float) -> List[float]:
        params = {"min": min_val, "max": max_val, "quantity": batch_size}
        try:
            resp = requests.get(f"{self.base_url}/double", headers=self.headers, params=params, timeout=10)
            resp.raise_for_status()
            return resp.json()["data"]
        except requests.RequestException as e:
            print(f"Double batch request failed: {e}")
            return []

    def get_random_doubles_batched(
        self,
        quantity: int,
        min_val: float = 0.0,
        max_val: float = 1.0
    ) -> List[float]:
        """
        Retrieve 'quantity' true-random doubles in [min_val, max_val), batching to respect API limits.
        """
        full_batches = quantity // self.max_double_batch
        remainder = quantity % self.max_double_batch
        result: List[float] = []
        for _ in range(full_batches):
            result.extend(self._get_double_batch(self.max_double_batch, min_val, max_val))
        if remainder:
            result.extend(self._get_double_batch(remainder, min_val, max_val))
        return result

        


In [5]:
if __name__ == "__main__":
    API_KEY = "aTo4BKRvnc49uRWDk034zaua87vGRXKk9TMLdfkI"
    qrng = QRNGaaS(API_KEY)

    # Example: get 100 doubles in batches
    doubles = qrng.get_random_doubles_batched(quantity=100, min_val=-1.0, max_val=1.0)
    print("Random doubles (sample):", doubles[::])

    #Example: get 100 ints in batches
    ints = qrng.get_random_ints_batched(quantity=100)
    print("Random ints (sample):", ints[::])

Double batch request failed: HTTPSConnectionPool(host='qrng.idqloud.com', port=443): Read timed out. (read timeout=10)
Random doubles (sample): [0.9742247491765068, -0.5747966249089977, 0.15903809674811287, 0.023676799725717013, 0.9407766719915487, -0.10894088225909226, 0.855798372878884, 0.4628708976942435, -0.2466066267271727, -0.6449593114245578, 0.8572215366727516, -0.1330699777673101, 0.805900604957847, 0.49591637028770563, -0.6758404218432095, 0.6435410201199092, -0.4342395510802399, 0.6087342787810668, -0.9444186996879835, 0.14086710072223352, 0.3116874210016747, 0.12894292060630086, 0.2146975635433146, 0.6449164241939029, 0.4667675141255767, -0.1395043285832931, -0.7556124883391955, 0.3853776605153656, 0.9995905444611011, -0.5483001750987357, 0.29871055486602294, 0.6054467655170637, -0.9092002197648228, -0.14708622003886673, 0.06470289382916383, 0.2522329166207997, -0.3452680368281065, -0.7646768005242828, -0.558922764143214, -0.7219815473975115, -0.87058352518225, -0.304809226